# Setup

## Import Libraries

In [71]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import requests
import time
import pandas as pd

## Open browser for scraping

In [72]:
# Set path for chromedriver
executable_path = {'chromedriver.exe'}
# Open browser
browser = Browser('chrome', headless=False)

In [73]:
# Go to the marathon results page
url = 'https://results.nyrr.org/event/M2019/finishers'
browser.visit(url)
time.sleep(1)

In [88]:
# Open up the full results list
completion_tracker = 0

while completion_tracker == 0:
    try:
        browser.find_by_xpath("/html/body/div[1]/div[2]/div/main/div/div[2]/div[2]/div/div/div[4]/a").click()
        print('Clicked!')
        time.sleep(1)
    except ElementDoesNotExist:
        completion_tracker = 1
        
print("Page Load Complete")

Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
Clicked!
C

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=87.0.4280.141)


In [87]:
# winner, winner!
browser.find_by_xpath("/html/body/div[1]/div[2]/div/main/div/div[2]/div[2]/div/div/div[4]/a").click()

In [2]:
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

NameError: name 'browser' is not defined

## Identify Data

One small issue we have is that not all runners have all data fields. Depending on where they come from, they may not have a city or state. Only runners from the USA have the city and state. It would be really nice to have the city and state for mapping runners' origin points within the US.

In [1]:
finisher_records = soup.find_all('div', {'ng-repeat': 'eventFinisher in eventFinishers'})

NameError: name 'soup' is not defined

In [91]:
finisher_records[500]

<div class="cmd-finisher ng-scope" ng-repeat="eventFinisher in eventFinishers">
<div class="container-fluid b-runners_detailes-list">
<div class="row rms-grid-item" ng-class="{ 'narrow' : layoutScope.isGridRowNarrow &amp;&amp; layoutScope.isDesktopView }" ng-click="openRunnerResults(eventCode, eventFinisher.bib)">
<div class="col-xs-4 col-sm-4 col-md-6 h-100" style="padding-right: 0;">
<div class="name rms-grid-line ng-binding">
                                Miguel Matta
                            </div>
<div class="details rms-grid-line">
<!-- ngIf: eventFinisher.gender || eventFinisher.age --><span class="ng-binding ng-scope" ng-if="eventFinisher.gender || eventFinisher.age">M35</span><!-- end ngIf: eventFinisher.gender || eventFinisher.age -->
<!-- ngIf: eventFinisher.city && eventFinisher.showUsa --><span class="ng-binding ng-scope" ng-if="eventFinisher.city &amp;&amp; eventFinisher.showUsa">Arlington</span><!-- end ngIf: eventFinisher.city && eventFinisher.showUsa -->
<!-- ngIf

In [92]:
len(finisher_records)

9996

In [6]:
finisher_record = finisher_records[5]

### Finisher Name

In [7]:
# This finds the first finisher name on the page.
finisher_name_first_method = soup.find_all(class_='name rms-grid-line ng-binding')[0].text

In [8]:
print(finisher_name_first_method)


                                Geoffrey Kamworor
                            


In [9]:
# This finds the finisher name in the selected finisher record. Slightly different approach, but important for making sure
# that we don't trip over missing data.
finisher_name = finisher_record.find(class_='name rms-grid-line ng-binding').text

In [10]:
print(finisher_name)


                                Jared Ward
                            


### Time

In [11]:
finisher_time = finisher_record.find_all(class_='result')[1].text[4:]
print(finisher_time)

2:10:45


### Pace

In [12]:
finisher_pace = finisher_record.find_all(class_='result')[2].text[4:]
print(finisher_pace)

05:00


### Place

In [13]:
finisher_place = finisher_record.find_all(class_='result')[3].text[5:]
print(finisher_place)

6


### Gender/Age

In [14]:
#finisher_gender_age = soup.find_all('span',{'ng-if':"eventFinisher.gender || eventFinisher.age"})
finisher_gender_age = finisher_record.find('span',{'ng-if':"eventFinisher.gender || eventFinisher.age"}).text

In [15]:
print(finisher_gender_age)

M31


### Bib

In [16]:
finisher_bib = finisher_record.find('span',{'ng-if':"eventFinisher.bib"}).text[4:]

In [17]:
print(finisher_bib)

6


### Country

In [18]:
finisher_country = finisher_record.find('span',{'ng-if':"eventFinisher.iaaf"}).text

In [19]:
print(finisher_country)

USA


### State/Province

In [20]:
try:
    finisher_state = finisher_record.find('span',{'ng-if':"eventFinisher.stateProvince && eventFinisher.showUsa"}).text
except:
    finisher_state = 0

In [21]:
print(finisher_state)

UT


### City

In [22]:
try:
    finisher_city = finisher_record.find('span',{'ng-if':"eventFinisher.city && eventFinisher.showUsa"}).text
except:
    finisher_city = 0

In [23]:
finisher_city

'Mapleton'

## Loop Construction

In [35]:
for record in finisher_records:
    finisher_name = record.find(class_='name rms-grid-line ng-binding').text
    finisher_time = record.find_all(class_='result')[1].text[4:]
    finisher_pace = record.find_all(class_='result')[2].text[4:]
    finisher_place = record.find_all(class_='result')[3].text[5:]
    finisher_gender_age = record.find('span',{'ng-if':"eventFinisher.gender || eventFinisher.age"}).text
    finisher_bib = record.find('span',{'ng-if':"eventFinisher.bib"}).text[4:]
    finisher_country = record.find('span',{'ng-if':"eventFinisher.iaaf"}).text
    try:
        finisher_state = record.find('span',{'ng-if':"eventFinisher.stateProvince && eventFinisher.showUsa"}).text
    except:
        finisher_state = 0
    try:
        finisher_city = record.find('span',{'ng-if':"eventFinisher.city && eventFinisher.showUsa"}).text
    except:
        finisher_city = 0
    print("---------------------")
    print(finisher_name)
    print(finisher_place)
    print(finisher_time)
    print(finisher_pace)
    print(finisher_gender_age)
    print(finisher_bib)
    print(finisher_country)
    print(finisher_state)
    print(finisher_city)

---------------------

                                Geoffrey Kamworor
                            
1
2:08:13
04:54
M26
3
KEN
0
0
---------------------

                                Albert Korir
                            
2
2:08:36
04:55
M25
5
KEN
0
0
---------------------

                                Girma Bekele Gebre
                            
3
2:08:38
04:55
M26
443
ETH
0
0
---------------------

                                Tamirat Tola
                            
4
2:09:20
04:56
M28
4
ETH
0
0
---------------------

                                Shura Kitata
                            
5
2:10:39
04:59
M23
2
ETH
0
0
---------------------

                                Jared Ward
                            
6
2:10:45
05:00
M31
6
USA
UT
Mapleton
---------------------

                                Stephen Sambu
                            
7
2:11:11
05:01
M31
10
KEN
0
0
---------------------

                                Yoshiki Takenouchi
                

M33
2110
USA
NY
Brooklyn
---------------------

                                Jurrien Adriaan Machiel van Rooden
                            
110
2:34:26
05:54
M32
1368
NED
0
0
---------------------

                                Jann Post
                            
111
2:34:29
05:54
M39
545
NOR
0
0
---------------------

                                Shawn Wiler
                            
112
2:34:29
05:54
M25
1380
USA
PA
Columbus
---------------------

                                Gregory Clark
                            
113
2:34:30
05:54
M28
560
USA
NY
New York
---------------------

                                Trond Smedsrud
                            
114
2:34:34
05:54
M37
1035
NOR
0
0
---------------------

                                Matthew Meyer
                            
115
2:34:40
05:54
M27
573
USA
NY
Astoria
---------------------

                                Luc Bourdon
                            
116
2:34:42
05:54
M31
2641
FRA
0
0
----------

# New Approach